In [46]:
# Local variables e imports necesarios
API_KEY = 'caf34c96749ce7ee8150ec8f591bac1b'  

from dbnomics import fetch_series, fetch_series_by_api_link
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import warnings
from plotly.offline import init_notebook_mode, iplot
import plotly.figure_factory as ff
import cufflinks as cl
import plotly.io as pio
import quandl as q 
from fredapi import Fred
import plotly.graph_objects as go

fred = Fred(API_KEY)
cl.go_offline()


In [21]:
ayer = datetime.now() - timedelta(days=1)
# Restar un año a la fecha de ayer
hace_un_ano = ayer - timedelta(days=365*2)
# Formatear la fecha en 'YYYY-MM-DD'
start_date = hace_un_ano.strftime('%Y-%m-%d')

In [13]:
#PMI

df1 = fetch_series('ISM/pmi/pm')
df1['period'] = pd.to_datetime(df1['period'], format = "%Y-%m-%d")
df1.iplot(kind= 'scatter',x='period',y='original_value', title = "PMI")

In [43]:
# Modelo 1: 3-month T-Bill Yields, 12-month ROC
df = {}
df['3-Month Treasury Bill Yield'] = fred.get_series(series_id='DGS3MO',observation_start=start_date)
# Considerando que los datos son diarios, 12 meses ≈ 252 días hábiles
df['3-Month T-Bill ROC 12m'] = df['3-Month Treasury Bill Yield'].pct_change(periods=252) * 100
df['SP500'] = fred.get_series(series_id='SP500',observation_start=start_date)
df['SP500 (Normalized)'] = (df['SP500'] - df['SP500'].iloc[0]) / df['SP500'].iloc[0] * 100
df = pd.DataFrame(df)
df[['SP500 (Normalized)','3-Month T-Bill ROC 12m']].iplot(kind='scatter', title = 'Si el Yield sube levente (6%) o baja, es bueno para equities', hline=6)


C:\Users\ftgon\AppData\Local\Temp\ipykernel_14208\3014959773.py:5: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [54]:
# Modelo 2: Dow Jones 20 Bond Index 12-Month ROC
df_2 = {}
df_2['AAA Corporate Bonds'] = fred.get_series(series_id='DAAA',observation_start=start_date)
# Considerando que los datos son diarios, 12 meses ≈ 252 días hábiles
df_2['AAA Corporate Bonds ROC 12m'] = df_2['AAA Corporate Bonds'].pct_change(periods=252) * 100
df_2['SP500'] = df['SP500'].copy()
df_2['SP500 (Normalized)'] = df['SP500 (Normalized)'].copy()
df_2 = pd.DataFrame(df_2)

df_2[['AAA Corporate Bonds ROC 12m','SP500 (Normalized)',]].iplot(kind='scatter', title = 'Si el sube o baja levemente (-1,5%), es bueno para equities', hline=-1.5)


C:\Users\ftgon\AppData\Local\Temp\ipykernel_14208\1108160987.py:5: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [57]:
# Modelo 3: Annual Change in 30-Year Goverment Bond Yield
df_3 = {}
df_3['30-Year Treasury Bill Yield'] = fred.get_series(series_id='DGS30',observation_start=start_date)
# Considerando que los datos son diarios, 12 meses ≈ 252 días hábiles
df_3['30-Year T-Bill ROC 12m'] = df_3['30-Year Treasury Bill Yield'].pct_change(periods=252) * 100
df_3['SP500'] = df['SP500'].copy()
df_3['SP500 (Normalized)'] = df['SP500 (Normalized)'].copy()
df_3 = pd.DataFrame(df_3)
df_3[['SP500 (Normalized)','30-Year T-Bill ROC 12m']].iplot(kind='scatter', title = 'Si el Yield sube levente (9%) o baja, es bueno para equities', hline=9)


C:\Users\ftgon\AppData\Local\Temp\ipykernel_14208\359729225.py:5: FutureWarning:

The default fill_method='pad' in Series.pct_change is deprecated and will be removed in a future version. Either fill in any non-leading NA values prior to calling pct_change or specify 'fill_method=None' to not fill NA values.



In [111]:
# Modelo 4: 30-Year Bond Yield vs 3-Month T-Bill Yield Curve Ratio
df_4 = {}
df_4['30Y v 3M Curve'] = (df_3['30-Year Treasury Bill Yield']/df['3-Month Treasury Bill Yield'])*100
df_4['SP500'] = df['SP500'].copy()
df_4['SP500 (Normalized)'] = df['SP500 (Normalized)'].copy()
df_4 = pd.DataFrame(df_4)
df_4.fillna(method='ffill', inplace=True)
df_4['30Y v 3M Curve (Normalized)'] = (df_4['30Y v 3M Curve'] - df_4['30Y v 3M Curve'].iloc[0]) / df_4['30Y v 3M Curve'].iloc[0] * 100
df_4['Curve Spread Moving Average'] = (df_4['30Y v 3M Curve (Normalized)'].rolling(window=20).mean())
df_4[['SP500 (Normalized)','30Y v 3M Curve (Normalized)','Curve Spread Moving Average']].iplot(kind='scatter', title = 'Si aumenta la curva entre bonos (1,15), es bueno para las equities')


C:\Users\ftgon\AppData\Local\Temp\ipykernel_14208\1243666654.py:7: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



In [104]:
#TODO SMA 40W BOND PRICES + SMA 40W INDEX PRICE 

,30Y v 3M Curve,SP500,SP500 (Normalized),30Y v 3M Curve (Normalized),Curve Spread Moving Average,Curve ROC
2022-08-08,113.207547,4140.06,0.000000,-0.000000,NaN,NaN
2022-08-09,112.734082,4122.47,-0.424873,0.418227,NaN,NaN
2022-08-10,114.716981,4210.24,1.695145,-1.333333,NaN,NaN
2022-08-11,120.229008,4207.27,1.623406,-6.202290,NaN,NaN
2022-08-12,118.631179,4280.15,3.383767,-4.790875,NaN,NaN
2022-08-15,113.970588,4297.14,3.794148,-0.674020,NaN,NaN
2022-08-16,115.185185,4305.20,3.988831,-1.746914,NaN,NaN
2022-08-17,117.537313,4274.04,3.236185,-3.824627,NaN,NaN
2022-08-18,115.867159,4283.74,3.470481,-2.349323,NaN,NaN
2022-08-19,117.518248,4228.48,2.135718,-3.807786,NaN,NaN
